In [1]:
#import necessary packages
import pandas as pd
from fuzzywuzzy import fuzz, process


C:\Users\giuli\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#import data
#season 2019_20
PL2019_ratings = pd.read_csv('Data/2019-20/Players/PL2019_ratings.csv')
PL2019_fl = pd.read_csv('Data/2019-20/Players/FL_19.csv')
PL19_lineups = pd.read_csv('Data/2019-20/Players/PL2019_lineups.csv')
PL2019_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2019_fl['player'] = PL2019_fl['name']
#season 2020_21
PL2020_ratings = pd.read_csv('Data/2020-21/Players/PL2020_ratings.csv')
PL2020_fl = pd.read_csv('Data/2020-21/Players/FL_20.csv')
PL20_lineups = pd.read_csv('Data/2020-21/Players/PL2020_lineups.csv')
PL2020_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2020_fl['player'] = PL2020_fl['name']
#season 2020_21
PL2021_ratings = pd.read_csv('Data/2021-22/Players/PL2021_ratings.csv')
PL2021_fl = pd.read_csv('Data/2021-22/Players/FL_21.csv')
PL21_lineups = pd.read_csv('Data/2021-22/Players/PL2021_lineups.csv')
PL2021_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2021_fl['player'] = PL2020_fl['name']


In [52]:
PL2020_fl.replace(['Brighton', 'Leeds', 'Leicester', 'Man City', 'Man Utd', 'Newcastle', 'Sheffield Utd','Spurs','West Brom','West Ham', 'Wolves'], ['Brighton & Hove Albion', 'Leeds United', 'Leicester City','Manchester City', 'Manchester United', 'Newcastle United', 'Sheffield United','Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], inplace=True)
PL2020_ratings.loc[PL2020_ratings['position'] == 'GK', 'category'] = 'Goalkeeper'

In [53]:
PL2020_ratings.to_excel('ratings_check.xlsx')

In [38]:
players_rating = PL2020_ratings[['player', 'Team']]
players_rating = players_rating.drop_duplicates()
players_rating

,player,Team
0,Joe Bryan,Fulham
1,Tim Ream,Fulham
2,Michael Hector,Fulham
3,Denis Odoi,Fulham
4,Ainsley Maitland-Niles,Arsenal
...,...,...
10190,João Virgínia,Everton
10203,John Ruddy,Wolverhampton Wanderers
10338,Scott Carson,Manchester City
10345,Jack Butland,Crystal Palace


In [39]:
players_fl = PL2020_fl[['player', 'name', 'team']]
players_fl = players_fl.drop_duplicates()
players_fl

,player,name,team
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion
1,Aaron Cresswell,Aaron Cresswell,West Ham United
2,Aaron Mooy,Aaron Mooy,Brighton & Hove Albion
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton
...,...,...,...
23955,William Fish,William Fish,Manchester United
24064,Sylvester Jasper,Sylvester Jasper,Fulham
24183,William Osula,William Osula,Sheffield United
24237,Tim Iroegbunam,Tim Iroegbunam,West Bromwich Albion


In [40]:
player_lineup = PL20_21_lineups[['player', 'team']]
player_lineup= PL20_21_lineups.drop_duplicates()
player_lineup['name_lineup'] = PL20_21_lineups['player']
player_lineup

,player,team,name_lineup
0,Marek Rodák,Fulham,Marek Rodák
1,Michael Hector,Fulham,Michael Hector
2,Denis Odoi,Fulham,Denis Odoi
3,Neeskens Kebano,Fulham,Neeskens Kebano
4,Tom Cairney,Fulham,Tom Cairney
...,...,...,...
14441,Hannibal Mejbri,Manchester United,Hannibal Mejbri
14442,William Thomas Fish,Manchester United,William Thomas Fish
14523,Tim Iroegbunam,West Bromwich Albion,Tim Iroegbunam
14541,Tyrese Francois,Fulham,Tyrese Francois


In [41]:
players_total = pd.merge(players_fl,player_lineup  , on='player', how='left')
#players_total['test'].isna().sum()
players_total = players_total.sort_values(by='name_lineup')

players_total

,player,name,team_x,team_y,name_lineup
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion,Brighton & Hove Albion,Aaron Connolly
1,Aaron Cresswell,Aaron Cresswell,West Ham United,West Ham United,Aaron Cresswell
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United,Sheffield United,Aaron Ramsdale
435,Aaron Wan-Bissaka,Aaron Wan-Bissaka,Manchester United,Manchester United,Aaron Wan-Bissaka
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton,Everton,Abdoulaye Doucouré
...,...,...,...,...,...
697,Joseph Willock,Joseph Willock,Newcastle United,NaN,NaN
709,Kyle John,Kyle John,Everton,NaN,NaN
719,Jaden Philogene-Bidace,Jaden Philogene-Bidace,Aston Villa,NaN,NaN
723,Jesurun Rak-Sakyi,Jesurun Rak-Sakyi,Crystal Palace,NaN,NaN


In [21]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1
players_merged = fuzzy_merge(player_lineup, players_fl ,'player', 'player', threshold=90, limit=1)
players_merged

,player,team,name_lineup,matches
0,Marek Rodák,Fulham,Marek Rodák,Marek Rodák
1,Michael Hector,Fulham,Michael Hector,Michael Hector
2,Denis Odoi,Fulham,Denis Odoi,Denis Odoi
3,Neeskens Kebano,Fulham,Neeskens Kebano,Neeskens Kebano
4,Tom Cairney,Fulham,Tom Cairney,Tom Cairney
...,...,...,...,...
14441,Hannibal Mejbri,Manchester United,Hannibal Mejbri,Hannibal Mejbri
14442,William Thomas Fish,Manchester United,William Thomas Fish,
14523,Tim Iroegbunam,West Bromwich Albion,Tim Iroegbunam,Tim Iroegbunam
14541,Tyrese Francois,Fulham,Tyrese Francois,Tyrese Francois


In [22]:
#players_merged = players_merged[['player','Team', 'matches']]
players_merged= players_merged.sort_values(by='matches')
players_merged.reset_index(inplace=True, drop=True)
players_merged.rename(columns={"matches": "name"}, inplace=True)
players_merged



,player,team,name_lineup,name
0,Kostas Tsimikas,Liverpool,Kostas Tsimikas,
1,Kiko Casilla,Leeds United,Kiko Casilla,
2,Khanya Leshabela,Leicester City,Khanya Leshabela,
3,Carlos Vinícius,Tottenham Hotspur,Carlos Vinícius,
4,Hélder Costa,Leeds United,Hélder Costa,
...,...,...,...,...
648,Youri Tielemans,Leicester City,Youri Tielemans,Youri Tielemans
649,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Yves Bissouma
650,Zack Steffen,Manchester City,Zack Steffen,Zack Steffen
651,Zak Brunt,Sheffield United,Zak Brunt,Zak Brunt


In [13]:
players_fl

,player,name,team
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion
1,Aaron Cresswell,Aaron Cresswell,West Ham United
2,Aaron Mooy,Aaron Mooy,Brighton & Hove Albion
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton
...,...,...,...
23955,William Fish,William Fish,Manchester United
24064,Sylvester Jasper,Sylvester Jasper,Fulham
24183,William Osula,William Osula,Sheffield United
24237,Tim Iroegbunam,Tim Iroegbunam,West Bromwich Albion


In [42]:
out = pd.merge(players_merged,players_fl , on='name', how='left')

out = out[['player_x', 'team_x', 'name', 'team_y']]
out.rename(columns={"player_x": "player_rating", "team_x": "team_rating", "name": "player_fl", "team_y": "team_fl" }, inplace=True)
out

,player_rating,team_rating,player_fl,team_fl
0,Kostas Tsimikas,Liverpool,,NaN
1,Kiko Casilla,Leeds United,,NaN
2,Khanya Leshabela,Leicester City,,NaN
3,Carlos Vinícius,Tottenham Hotspur,,NaN
4,Hélder Costa,Leeds United,,NaN
...,...,...,...,...
672,Youri Tielemans,Leicester City,Youri Tielemans,Leicester City
673,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Brighton & Hove Albion
674,Zack Steffen,Manchester City,Zack Steffen,Manchester City
675,Zak Brunt,Sheffield United,Zak Brunt,Sheffield United


In [26]:
out.to_excel('names_2020.xlsx')

In [100]:
name_matching = pd.read_excel('name_matching_2019.xlsx')
name_matching

,Unnamed: 0,player_rating,team_rating,player_fl
0,55,Aaron Connolly,Brighton & Hove Albion,Aaron Connolly
1,56,Aaron Cresswell,West Ham United,Aaron Cresswell
2,57,Aaron Lennon,Burnley,Aaron Lennon
3,58,Aaron Mooy,Brighton & Hove Albion,Aaron Mooy
4,59,Aaron Ramsdale,Bournemouth,Aaron Ramsdale
...,...,...,...,...
616,614,Yerry Mina,Everton,Yerry Mina
617,615,Yoshinori Mutō,Newcastle United,Yoshinori Muto
618,616,Youri Tielemans,Leicester City,Youri Tielemans
619,617,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma


In [101]:
name_matching = name_matching[['player_rating', 'player_fl']]
name_matching = name_matching.drop_duplicates()
name_matching.rename(columns={"player_fl": "player"}, inplace=True)
name_matching

,player_rating,player
0,Aaron Connolly,Aaron Connolly
1,Aaron Cresswell,Aaron Cresswell
2,Aaron Lennon,Aaron Lennon
3,Aaron Mooy,Aaron Mooy
4,Aaron Ramsdale,Aaron Ramsdale
...,...,...
616,Yerry Mina,Yerry Mina
617,Yoshinori Mutō,Yoshinori Muto
618,Youri Tielemans,Youri Tielemans
619,Yves Bissouma,Yves Bissouma


In [102]:
PL2019_fl ['date'] = PL2019_fl['kickoff_time'].str.slice(start=0, stop=10)
PL2019_fl

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player,date
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,0,0,0,0,50,True,0,1,Aaron Cresswell,2019-08-10
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,1,0,0,0,50,True,0,1,Aaron Lennon,2019-08-10
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,0,0,0,0,50,False,0,1,Aaron Mooy,2019-08-10
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,2,0,0,0,45,True,0,1,Aaron Ramsdale,2019-08-10
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,8,0,0,0,55,True,0,1,Aaron Wan-Bissaka,2019-08-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Youri Tielemans,0,0,11,0,37.4,448,376,1,0,...,2,-3103,1905,5008,61,True,0,47,Youri Tielemans,2020-07-26
22556,Yves Bissouma,0,0,22,0,1.3,53,372,1,1,...,7,77,147,70,50,False,0,47,Yves Bissouma,2020-07-26
22557,Zech Medley,0,0,0,0,0.0,636,371,0,0,...,0,153,325,172,40,True,0,47,Zech Medley,2020-07-26
22558,Çaglar Söyüncü,0,0,0,0,0.0,164,376,0,0,...,0,-95515,165,95680,49,True,0,47,Çaglar Söyüncü,2020-07-26


In [105]:
PL2019_fl['player'] = PL2019_fl['player'].str.strip()
name_matching['player'] = name_matching['player'].str.strip()
PL2019_fl_matched = pd.merge(PL2019_fl, name_matching , on='player', how='left')
#PL2020_fl_matched['GW']= PL2020_fl_matched['GW'].astype(float)
PL2019_fl_matched['ID'] = PL2019_fl_matched['player_rating'] + '_' + PL2019_fl_matched['date'].astype(str)
PL2019_fl_matched

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player,date,player_rating,ID
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,0,0,50,True,0,1,Aaron Cresswell,2019-08-10,Aaron Cresswell,Aaron Cresswell_2019-08-10
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,0,0,50,True,0,1,Aaron Lennon,2019-08-10,Aaron Lennon,Aaron Lennon_2019-08-10
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,0,0,50,False,0,1,Aaron Mooy,2019-08-10,Aaron Mooy,Aaron Mooy_2019-08-10
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,0,0,45,True,0,1,Aaron Ramsdale,2019-08-10,Aaron Ramsdale,Aaron Ramsdale_2019-08-10
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,0,0,55,True,0,1,Aaron Wan-Bissaka,2019-08-11,Aaron Wan-Bissaka,Aaron Wan-Bissaka_2019-08-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Youri Tielemans,0,0,11,0,37.4,448,376,1,0,...,1905,5008,61,True,0,47,Youri Tielemans,2020-07-26,Youri Tielemans,Youri Tielemans_2020-07-26
22556,Yves Bissouma,0,0,22,0,1.3,53,372,1,1,...,147,70,50,False,0,47,Yves Bissouma,2020-07-26,Yves Bissouma,Yves Bissouma_2020-07-26
22557,Zech Medley,0,0,0,0,0.0,636,371,0,0,...,325,172,40,True,0,47,Zech Medley,2020-07-26,Zech Medley,Zech Medley_2020-07-26
22558,Çaglar Söyüncü,0,0,0,0,0.0,164,376,0,0,...,165,95680,49,True,0,47,Çaglar Söyüncü,2020-07-26,Çağlar Söyüncü,Çağlar Söyüncü_2020-07-26


In [106]:
PL2019_ratings['ID'] = PL2019_ratings['player'] + '_' + PL2019_ratings['date'].astype(str)
PL2019_ratings

,player,position,age,minutes,Team,matchweek,category,Rating,date,ID
0,Andrew Robertson,LB,25-151,90.0,Liverpool,1.0,Defender,38.324184,2019-08-09,Andrew Robertson_2019-08-09
1,Virgil van Dijk,CB,28-032,90.0,Liverpool,1.0,Defender,23.823938,2019-08-09,Virgil van Dijk_2019-08-09
2,Joe Gomez,CB,22-078,90.0,Liverpool,1.0,Defender,22.788262,2019-08-09,Joe Gomez_2019-08-09
3,Trent Alexander-Arnold,RB,20-306,90.0,Liverpool,1.0,Defender,32.156373,2019-08-09,Trent Alexander-Arnold_2019-08-09
4,Jamal Lewis,LB,21-196,90.0,Norwich City,1.0,Defender,38.828733,2019-08-09,Jamal Lewis_2019-08-09
...,...,...,...,...,...,...,...,...,...,...
10611,David de Gea,GK,29-262,90.0,Manchester United,38.0,Goalkeeper,55.802267,2020-07-26,David de Gea_2020-07-26
10612,Ederson,GK,26-344,90.0,Manchester City,38.0,Goalkeeper,57.421139,2020-07-26,Ederson_2020-07-26
10613,Tim Krul,GK,32-114,90.0,Norwich City,38.0,Goalkeeper,29.910324,2020-07-26,Tim Krul_2020-07-26
10614,Alex McCarthy,GK,30-236,90.0,Southampton,38.0,Goalkeeper,40.027284,2020-07-26,Alex McCarthy_2020-07-26


In [107]:
PL2019_matched = pd.merge(PL2019_fl_matched, PL2019_ratings , on='ID', how='left')
PL2019_matched

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,ID,player_y,position,age,minutes_y,Team,matchweek_y,category,Rating,date_y
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,Aaron Cresswell_2019-08-10,Aaron Cresswell,LB,29-238,90.0,West Ham United,1.0,Defender,16.471345,2019-08-10
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,Aaron Lennon_2019-08-10,Aaron Lennon,LM,32-116,7.0,Burnley,1.0,Midfield,2.007932,2019-08-10
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,Aaron Mooy_2019-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,Aaron Ramsdale_2019-08-10,Aaron Ramsdale,GK,21-088,90.0,Bournemouth,1.0,Goalkeeper,37.868320,2019-08-10
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,Aaron Wan-Bissaka_2019-08-11,Aaron Wan-Bissaka,RB,21-258,90.0,Manchester United,1.0,Defender,34.567219,2019-08-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22556,Youri Tielemans,0,0,11,0,37.4,448,376,1,0,...,Youri Tielemans_2020-07-26,Youri Tielemans,CM,23-080,72.0,Leicester City,38.0,Midfield,23.693602,2020-07-26
22557,Yves Bissouma,0,0,22,0,1.3,53,372,1,1,...,Yves Bissouma_2020-07-26,Yves Bissouma,DM,23-331,72.0,Brighton & Hove Albion,38.0,Midfield,10.941405,2020-07-26
22558,Zech Medley,0,0,0,0,0.0,636,371,0,0,...,Zech Medley_2020-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22559,Çaglar Söyüncü,0,0,0,0,0.0,164,376,0,0,...,Çağlar Söyüncü_2020-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
PL2019_matched['Rating'] = PL2019_matched['Rating'].fillna(0)
PL2019_matched

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,ID,player_y,position,age,minutes_y,Team,matchweek_y,category,Rating,date_y
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,Aaron Cresswell_2019-08-10,Aaron Cresswell,LB,29-238,90.0,West Ham United,1.0,Defender,16.471345,2019-08-10
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,Aaron Lennon_2019-08-10,Aaron Lennon,LM,32-116,7.0,Burnley,1.0,Midfield,2.007932,2019-08-10
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,Aaron Mooy_2019-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,Aaron Ramsdale_2019-08-10,Aaron Ramsdale,GK,21-088,90.0,Bournemouth,1.0,Goalkeeper,37.868320,2019-08-10
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,Aaron Wan-Bissaka_2019-08-11,Aaron Wan-Bissaka,RB,21-258,90.0,Manchester United,1.0,Defender,34.567219,2019-08-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22556,Youri Tielemans,0,0,11,0,37.4,448,376,1,0,...,Youri Tielemans_2020-07-26,Youri Tielemans,CM,23-080,72.0,Leicester City,38.0,Midfield,23.693602,2020-07-26
22557,Yves Bissouma,0,0,22,0,1.3,53,372,1,1,...,Yves Bissouma_2020-07-26,Yves Bissouma,DM,23-331,72.0,Brighton & Hove Albion,38.0,Midfield,10.941405,2020-07-26
22558,Zech Medley,0,0,0,0,0.0,636,371,0,0,...,Zech Medley_2020-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
22559,Çaglar Söyüncü,0,0,0,0,0.0,164,376,0,0,...,Çağlar Söyüncü_2020-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


In [110]:
PL2019_check = PL2019_matched[['player_x','name' ,'date_x' ,'Team', 'matchweek_x', 'category', 'minutes_x', 'minutes_y', 'Rating', 'bonus', 'bps',  'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home' ]]
PL2019_check

,player_x,name,date_x,Team,matchweek_x,category,minutes_x,minutes_y,Rating,bonus,...,ict_index,influence,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home
0,Aaron Cresswell,Aaron Cresswell,2019-08-10,West Ham United,1,Defender,90,90.0,16.471345,0,...,1.1,9.0,1.5,0.0,0,0,0,0,50,True
1,Aaron Lennon,Aaron Lennon,2019-08-10,Burnley,1,Midfield,6,7.0,2.007932,0,...,0.2,2.0,0.0,0.0,1,0,0,0,50,True
2,Aaron Mooy,Aaron Mooy,2019-08-10,NaN,1,NaN,0,NaN,0.000000,0,...,0.0,0.0,0.0,0.0,0,0,0,0,50,False
3,Aaron Ramsdale,Aaron Ramsdale,2019-08-10,Bournemouth,1,Goalkeeper,90,90.0,37.868320,0,...,1.0,9.8,0.0,0.0,2,0,0,0,45,True
4,Aaron Wan-Bissaka,Aaron Wan-Bissaka,2019-08-11,Manchester United,1,Defender,90,90.0,34.567219,2,...,4.9,30.4,16.1,2.0,8,0,0,0,55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22556,Youri Tielemans,Youri Tielemans,2020-07-26,Leicester City,47,Midfield,72,72.0,23.693602,0,...,6.9,12.6,37.4,19.0,2,-3103,1905,5008,61,True
22557,Yves Bissouma,Yves Bissouma,2020-07-26,Brighton & Hove Albion,47,Midfield,72,72.0,10.941405,0,...,4.8,39.2,1.3,7.0,7,77,147,70,50,False
22558,Zech Medley,Zech Medley,2020-07-26,NaN,47,NaN,0,NaN,0.000000,0,...,0.0,0.0,0.0,0.0,0,153,325,172,40,True
22559,Çaglar Söyüncü,Çaglar Söyüncü,2020-07-26,NaN,47,NaN,0,NaN,0.000000,0,...,0.0,0.0,0.0,0.0,0,-95515,165,95680,49,True


In [60]:
PL2019_matched.to_excel('check_2019.xlsx')

In [73]:
PL2020_total = PL2020_matched[['player_x','player_rating','date_x' ,'team','Team' ,'category','position_x', 'minutes_x' ,'minutes_y','Rating', 'xP', 'bonus', 'bps',  'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home', 'opponent_team', 'selected' ]]
PL2020_total.rename(columns={"player_x": "player_fl","player_rating": "player" ,"date_x": "date", "Team": "team_rating","team":"team",  "position_x": "position_fl", "minutes_x": "minutes_fl","minutes_y": "minutes_rating" ,"Rating": "rating"}, inplace=True)
PL2020_total


C:\Users\giuli\AppData\Local\Temp\ipykernel_100908\3791840291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total.rename(columns={"player_x": "player_fl","player_rating": "player" ,"date_x": "date", "Team": "team_rating","team":"team",  "position_x": "position_fl", "minutes_x": "minutes_fl","minutes_y": "minutes_rating" ,"Rating": "rating"}, inplace=True)


,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,rating,...,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected
0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,Brighton & Hove Albion,Forward,FWD,45,46.0,20.877365,...,0.3,32.0,1,0,0,0,55,True,5,32205
1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,West Ham United,Defender,DEF,90,90.0,39.872009,...,11.2,0.0,1,0,0,0,50,True,14,70232
2,Aaron Mooy,0,2020-09-14,Brighton & Hove Albion,NaN,NaN,MID,0,NaN,0.000000,...,0.0,0.0,0,0,0,0,50,True,5,2010
3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,26.083953,...,0.0,0.0,1,0,0,0,50,True,20,462939
4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,Everton,Midfield,MID,90,90.0,31.205598,...,44.6,4.0,3,0,0,0,55,False,17,143040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,55.932138,...,10.0,0.0,9,13618,17381,3763,46,True,4,162985
24377,Adrien Silva,0,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0.0,0.0,0,-16,0,16,44,True,17,16396
24378,Sidnei Tavares,Sidnei Tavares,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0.0,0.0,0,32,60,28,44,True,17,337
24379,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,Leeds United,Midfield,MID,90,90.0,34.090387,...,12.8,8.0,6,210,789,579,49,True,18,74008


In [74]:
PL2020_total['player_ID'] = PL2020_total['player'].astype(str) + PL2020_total['team'] + PL2020_total['date'].astype(str)
PL2020_total['game_ID'] = PL2020_total['team'] + PL2020_total['date'].astype(str)

C:\Users\giuli\AppData\Local\Temp\ipykernel_100908\2868901141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total['player_ID'] = PL2020_total['player'].astype(str) + PL2020_total['team'] + PL2020_total['date'].astype(str)
C:\Users\giuli\AppData\Local\Temp\ipykernel_100908\2868901141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total['game_ID'] = PL2020_total['team'] + PL2020_total['date'].astype(str)


In [75]:
PL2020_total

,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,rating,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected,player_ID,game_ID
0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,Brighton & Hove Albion,Forward,FWD,45,46.0,20.877365,...,1,0,0,0,55,True,5,32205,Aaron ConnollyBrighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14
1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,West Ham United,Defender,DEF,90,90.0,39.872009,...,1,0,0,0,50,True,14,70232,Aaron CresswellWest Ham United2020-09-12,West Ham United2020-09-12
2,Aaron Mooy,0,2020-09-14,Brighton & Hove Albion,NaN,NaN,MID,0,NaN,0.000000,...,0,0,0,0,50,True,5,2010,0Brighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14
3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,26.083953,...,1,0,0,0,50,True,20,462939,Aaron RamsdaleSheffield United2020-09-14,Sheffield United2020-09-14
4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,Everton,Midfield,MID,90,90.0,31.205598,...,3,0,0,0,55,False,17,143040,Abdoulaye DoucouréEverton2020-09-13,Everton2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,55.932138,...,9,13618,17381,3763,46,True,4,162985,Aaron RamsdaleSheffield United2021-05-23,Sheffield United2021-05-23
24377,Adrien Silva,0,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0,-16,0,16,44,True,17,16396,0Leicester City2021-05-23,Leicester City2021-05-23
24378,Sidnei Tavares,Sidnei Tavares,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0,32,60,28,44,True,17,337,Sidnei TavaresLeicester City2021-05-23,Leicester City2021-05-23
24379,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,Leeds United,Midfield,MID,90,90.0,34.090387,...,6,210,789,579,49,True,18,74008,Kalvin PhillipsLeeds United2021-05-23,Leeds United2021-05-23


In [76]:
PL2020_total.to_csv('Data/2020-21/Players/PL2020_total.csv')